In [1]:
# !pip install -q gdown inference-gpu
# !pip install -q onnxruntime-gpu==1.18.0 --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
# !pip install -q git+https://github.com/roboflow/sports.git
# !pip uninstall -y supervision && pip install -q supervision>=0.23.0
# !pip install ultralytics

In [2]:
import cv2
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import supervision as sv
from google.colab import userdata
from inference import get_model
from ultralytics import YOLO

from supervision import Detections, BoxAnnotator, EllipseAnnotator, TriangleAnnotator, LabelAnnotator
from typing import Generator, Iterable, List, TypeVar
import umap
from sklearn.cluster import KMeans
from transformers import AutoProcessor, SiglipVisionModel
from sports.common.team import TeamClassifier
from sports.configs.soccer import SoccerPitchConfiguration
from sports.common.view import ViewTransformer
from sports.annotators.soccer import (
    draw_pitch,
    draw_points_on_pitch,
    draw_pitch_voronoi_diagram
)


ModelDependencyMissing: Your `inference` configuration does not support Qwen2.5-VL model. Use pip install 'inference[transformers]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[clip]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration does not support GroundingDINO model. Use pip install 'inference[grounding-dino]' to install missing requirements.
ModelDependencyMissing: Your `inference` configuration d

In [3]:
# Constants
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3
STRIDE = 50

source_video_path = "/kaggle/input/video-analysis/3.mp4"
output_video_path = "/kaggle/working/output_video.mp4"
ROBOFLOW_API_KEY = 'pQ2a96kNxTXy7kvwiXZe'


In [4]:
# Initialize Video Processing
frame_generator = sv.get_video_frames_generator(source_path=source_video_path)
video_info = sv.VideoInfo.from_video_path(source_video_path)
writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), video_info.fps, (video_info.width, video_info.height))

# Initialize Tracker
tracker = sv.ByteTrack()

# Initialize Team Classifier
team_classifier = TeamClassifier(device="cuda")

# Initialize Annotators
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=20, height=17
)

In [5]:
def get_center_of_bbox(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int((y1+y2)/2)

def get_bbox_width(bbox):
    return bbox[2]-bbox[0]

def measure_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

def measure_xy_distance(p1,p2):
    return p1[0]-p2[0],p1[1]-p2[1]

def get_foot_position(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int(y2)

In [6]:
def load_models(api_key: str) -> tuple:
    """
    Load player and field detection models.

    Args:
        api_key (str): The API key for accessing the Roboflow model.

    Returns:
        tuple: A tuple containing the loaded player detection model.
    """
    # Load the player detection model using the Roboflow API
    player_detection_model = get_model(model_id="football-players-detection-3zvbc/11", api_key=api_key)
    
    # Return the loaded model
    return player_detection_model

In [7]:
def resolve_goalkeepers_team_id(players: Detections, goalkeepers: Detections) -> np.ndarray:
    """
    Resolve goalkeepers' team IDs based on their positions.

    Args:
        players (Detections): Detections object containing all players.
        goalkeepers (Detections): Detections object containing all goalkeepers.

    Returns:
        np.ndarray: An array of team IDs (0 or 1) for each goalkeeper.
    """
    # Get the bottom-center coordinates of all goalkeepers
    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    
    # Get the bottom-center coordinates of all players
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    
    # Calculate the centroid (mean position) of players in team 0
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    
    # Calculate the centroid (mean position) of players in team 1
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)
    
    # Initialize a list to store team IDs for goalkeepers
    goalkeepers_team_id = []
    
    # Assign team IDs to goalkeepers based on their proximity to team centroids
    for goalkeeper_xy in goalkeepers_xy:
        # Calculate the Euclidean distance between the goalkeeper and team 0's centroid
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        
        # Calculate the Euclidean distance between the goalkeeper and team 1's centroid
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        
        # Assign the goalkeeper to the team whose centroid is closer
        goalkeepers_team_id.append(0 if dist_0 < dist_1 else 1)
    
    # Return the team IDs as a NumPy array
    return np.array(goalkeepers_team_id)

In [8]:
def process_frame_for_detection_and_tracking(frame, player_detection_model, team_classifier, tracker):
    """
    Process a single frame to detect and classify objects (players, referees, ball).

    Args:
        frame: The input video frame.
        player_detection_model: The model used to detect players, referees, and the ball.
        team_classifier: The model used to classify players into teams.
        tracker: The object tracker (e.g., ByteTrack) to maintain consistent IDs across frames.

    Returns:
        tuple: A tuple containing the annotated frame and the detections.
    """
    # Detect objects in the frame using the player detection model
    result = player_detection_model.infer(frame, confidence=0.3)[0]
    detections = Detections.from_inference(result)

    # Separate Ball and Other Objects
    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)  # Add padding to ball bounding boxes
    all_detections = detections[detections.class_id != BALL_ID]  # Exclude the ball from other detections
    all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)  # Apply Non-Maximum Suppression
    all_detections = tracker.update_with_detections(detections=all_detections)  # Update tracks using the tracker

    # Split Detections into Players, Goalkeepers, and Referees
    goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
    players_detections = all_detections[all_detections.class_id == PLAYER_ID]
    referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

    # Team Assignment
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]  # Crop player regions
    players_detections.class_id = team_classifier.predict(players_crops)  # Classify players into teams
    goalkeepers_detections.class_id = resolve_goalkeepers_team_id(players_detections, goalkeepers_detections)  # Assign team IDs to goalkeepers
    referees_detections.class_id -= 1  # Adjust referee class IDs (if needed)

    # Merge All Detections
    all_detections = Detections.merge([players_detections, goalkeepers_detections, referees_detections])
    labels = [f"#{tracker_id}" for tracker_id in all_detections.tracker_id]  # Create labels for each tracked object
    all_detections.class_id = all_detections.class_id.astype(int)  # Ensure class IDs are integers

    # Annotate Frame
    annotated_frame = frame.copy()
    annotated_frame = ellipse_annotator.annotate(scene=annotated_frame, detections=all_detections)  # Draw ellipses around players
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=all_detections, labels=labels)  # Add labels (e.g., player IDs)
    annotated_frame = triangle_annotator.annotate(scene=annotated_frame, detections=ball_detections)  # Draw triangles for the ball

    return annotated_frame, all_detections

In [9]:
# Collect and Train Team Classifier
def collect_and_train_team_classifier(
    source_video_path: str,
    player_detection_model,
    team_classifier,
    stride: int = STRIDE
) -> None:
    """
    Collect player crops from the video and train the team classifier.

    Args:
        source_video_path (str): Path to the source video.
        player_detection_model: Model for detecting players.
        team_classifier: Team classifier to be trained.
        stride (int): Stride for frame sampling. Defaults to STRIDE.
    """
    # Initialize frame generator
    frame_generator = sv.get_video_frames_generator(source_path=source_video_path, stride=stride)
    video_info = sv.VideoInfo.from_video_path(source_video_path)

    # Collect player crops for training the team classifier
    training_crops = []

    # First pass: Collect player crops for training
    for frame in tqdm(frame_generator, total=video_info.total_frames, desc='collecting crops'):
        result = player_detection_model.infer(frame, confidence=0.3)[0]
        detections = Detections.from_inference(result)
        players_detections = detections[detections.class_id == PLAYER_ID]
        player_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
        training_crops.extend(player_crops)

    # Train the team classifier
    print("Training Team Classifier...")
    team_classifier.fit(training_crops)
    print("Team Classifier training complete.")


In [10]:
def initialize_camera_movement_estimator(frame):
    """
    Initialize the camera movement estimator.

    Args:
        frame: The first frame of the video.

    Returns:
        tuple: A tuple containing:
            - lk_params: Parameters for the Lucas-Kanade optical flow algorithm.
            - features: Parameters for detecting features to track.
            - first_frame_grayscale: The first frame converted to grayscale.
    """
    # Parameters for the Lucas-Kanade optical flow algorithm
    lk_params = dict(
        winSize=(15, 15),  # Size of the search window for optical flow
        maxLevel=2,  # Maximum pyramid level for optical flow
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)  # Termination criteria
    )

    # Convert the first frame to grayscale
    first_frame_grayscale = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Create a mask to specify regions where features should be detected
    mask_features = np.zeros_like(first_frame_grayscale)
    mask_features[:, 0:20] = 1  # Enable feature detection in the leftmost 20 columns
    mask_features[:, 900:1050] = 1  # Enable feature detection in columns 900 to 1050

    # Parameters for detecting features to track
    features = dict(
        maxCorners=100,  # Maximum number of corners to detect
        qualityLevel=0.3,  # Minimum quality of corners (lower values detect more corners)
        minDistance=3,  # Minimum distance between detected corners
        blockSize=7,  # Size of the neighborhood for corner detection
        mask=mask_features  # Mask specifying where to detect corners
    )

    return lk_params, features, first_frame_grayscale

In [11]:
def get_camera_movement(frames, lk_params, features):
    """
    Estimate camera movement across frames using optical flow.

    Args:
        frames (list): List of video frames.
        lk_params (dict): Parameters for the Lucas-Kanade optical flow algorithm.
        features (dict): Parameters for detecting features to track.

    Returns:
        list: A list of camera movement vectors for each frame.
    """
    # Initialize camera movement list with zeros
    camera_movement = [[0, 0]] * len(frames)

    # Convert the first frame to grayscale
    old_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)

    # Detect features to track in the first frame
    old_features = cv2.goodFeaturesToTrack(old_gray, **features)

    # Iterate through frames to estimate camera movement
    for frame_num in range(1, len(frames)):
        # Convert the current frame to grayscale
        frame_gray = cv2.cvtColor(frames[frame_num], cv2.COLOR_BGR2GRAY)

        # Compute optical flow between the previous and current frame
        new_features, _, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, old_features, None, **lk_params)

        # Initialize variables to track maximum movement
        max_distance = 0
        camera_movement_x, camera_movement_y = 0, 0

        # Calculate movement for each feature point
        for i, (new, old) in enumerate(zip(new_features, old_features)):
            new_features_point = new.ravel()  # Flatten the new feature point coordinates
            old_features_point = old.ravel()  # Flatten the old feature point coordinates

            # Calculate the Euclidean distance between the old and new feature points
            distance = np.linalg.norm(new_features_point - old_features_point)

            # Update maximum movement if the current distance is larger
            if distance > max_distance:
                max_distance = distance
                camera_movement_x, camera_movement_y = old_features_point[0] - new_features_point[0], old_features_point[1] - new_features_point[1]

        # If the maximum movement exceeds the threshold, update the camera movement list
        if max_distance > 5:  # Minimum distance threshold
            camera_movement[frame_num] = [camera_movement_x, camera_movement_y]
            # Re-detect features in the current frame for the next iteration
            old_features = cv2.goodFeaturesToTrack(frame_gray, **features)

        # Update the previous frame for the next iteration
        old_gray = frame_gray.copy()

    return camera_movement

In [12]:
def draw_camera_movement(frame, camera_movement):
    """
    Draw camera movement information on a video frame.

    Args:
        frame: The input video frame.
        camera_movement (list): A list containing the camera movement in the X and Y directions.

    Returns:
        frame: The annotated frame with camera movement information.
    """
    # Create a copy of the frame to avoid modifying the original
    frame = frame.copy()

    # Create an overlay for the text background
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (500, 100), (255, 255, 255), -1)  # Draw a white rectangle

    # Blend the overlay with the frame to create a semi-transparent background
    alpha = 0.6  # Transparency factor
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    # Extract camera movement in the X and Y directions
    x_movement, y_movement = camera_movement

    # Add text to display the camera movement in the X direction
    frame = cv2.putText(
        frame,  # The frame to draw on
        f"Camera Movement X: {x_movement:.2f}",  # Text to display
        (10, 30),  # Position of the text (bottom-left corner)
        cv2.FONT_HERSHEY_SIMPLEX,  # Font type
        1,  # Font scale
        (0, 0, 0),  # Text color (black)
        3  # Thickness of the text
    )

    # Add text to display the camera movement in the Y direction
    frame = cv2.putText(
        frame,  # The frame to draw on
        f"Camera Movement Y: {y_movement:.2f}",  # Text to display
        (10, 60),  # Position of the text (below the X movement text)
        cv2.FONT_HERSHEY_SIMPLEX,  # Font type
        1,  # Font scale
        (0, 0, 0),  # Text color (black)
        3  # Thickness of the text
    )

    return frame

In [13]:
def initialize_view_transformer(pixel_vertices, target_vertices):
    """
    Initialize the perspective transformer for converting points from the video frame to a top-down view.

    Args:
        pixel_vertices (list): A list of four points in the video frame (e.g., corners of the pitch).
        target_vertices (list): A list of four points in the target top-down view (e.g., corners of a rectangle).

    Returns:
        np.ndarray: A 3x3 perspective transformation matrix.
    """
    # Compute the perspective transformation matrix
    perspective_transformer = cv2.getPerspectiveTransform(
        np.float32(pixel_vertices),  # Source points (video frame)
        np.float32(target_vertices)  # Target points (top-down view)
    )

    return perspective_transformer

In [14]:
def transform_point(pixel_vertices, point, perspective_transformer):
    """
    Transform a point from the video frame to the top-down view using the perspective transformer.

    Args:
        pixel_vertices (list): A list of four points defining the region of interest (ROI) in the video frame.
        point (np.ndarray): The point to transform, in the format [x, y].
        perspective_transformer (np.ndarray): The 3x3 perspective transformation matrix.

    Returns:
        np.ndarray: The transformed point in the top-down view, or None if the point is outside the ROI.
    """
    # Convert the point to integer coordinates
    p = (int(point[0]), int(point[1]))

    # Check if the point lies inside the region of interest (ROI)
    is_inside = cv2.pointPolygonTest(np.array(pixel_vertices, dtype=np.int32), p, False) >= 0

    # If the point is outside the ROI, return None
    if not is_inside:
        return None

    # Reshape the point for perspective transformation
    reshaped_point = point.reshape(-1, 1, 2).astype(np.float32)

    # Apply the perspective transformation
    transformed_point = cv2.perspectiveTransform(reshaped_point, perspective_transformer)

    # Reshape the transformed point to a 2D array and return it
    return transformed_point.reshape(-1, 2)

In [15]:
def add_transformed_positions_to_tracks(tracks, pixel_vertices, perspective_transformer):
    """
    Add transformed positions (top-down view) to the tracks dictionary.

    Args:
        tracks (dict): A dictionary containing tracks for players, referees, and the ball.
        pixel_vertices (list): A list of four points defining the region of interest (ROI) in the video frame.
        perspective_transformer (np.ndarray): The 3x3 perspective transformation matrix.

    Returns:
        dict: The updated tracks dictionary with transformed positions added.
    """
    # Iterate through each object (e.g., players, referees, ball) in the tracks dictionary
    for object, object_tracks in tracks.items():
        # Iterate through each frame for the current object
        for frame_num, track in enumerate(object_tracks):
            # Iterate through each track in the current frame
            for track_id, track_info in track.items():
                # Get the adjusted position of the object
                position = track_info.get('position_adjusted', None)

                # If the position is available, transform it to the top-down view
                if position is not None:
                    position = np.array(position)  # Convert the position to a NumPy array
                    position_transformed = transform_point(pixel_vertices, position, perspective_transformer)

                    # If the transformed position is valid, add it to the tracks dictionary
                    if position_transformed is not None:
                        position_transformed = position_transformed.squeeze().tolist()  # Convert to a list
                    tracks[object][frame_num][track_id]['position_transformed'] = position_transformed

    return tracks

In [16]:
def initialize_tracker(model_path):
    """
    Initialize the YOLO model and ByteTrack tracker.

    Args:
        model_path (str): Path to the YOLO model weights file.

    Returns:
        tuple:  the YOLO model and ByteTrack tracker.
    """
    # Load the YOLO model from the specified path
    model = YOLO(model_path)

    # Initialize the ByteTrack tracker
    tracker = sv.ByteTrack()

    return model, tracker

In [17]:
def get_object_tracks(model, tracker, frames):
    """
    Get object tracks from frames using the YOLO model and ByteTrack tracker.

    Args:
        model: The YOLO model for object detection.
        tracker: The ByteTrack tracker for object tracking.
        frames (list): List of video frames.
        
    Returns:
        dict: A dictionary containing tracks for players, referees, and the ball.
    """
    """
    Get object tracks from frames using a YOLO model and a tracker.

    Args:
        model: The YOLO model used for object detection.
        tracker: The tracker object used for tracking detected objects across frames.
        frames (list): A list of frames (images) to process.
        read_from_stub (bool, optional): Whether to load precomputed tracks from a stub file. Defaults to False.
        stub_path (str, optional): Path to the stub file containing precomputed tracks. Defaults to None.

    Returns:
        dict: A dictionary containing tracks for players, referees, and the ball.
              Each key maps to a list of dictionaries representing tracked objects in each frame.
    """
    # Otherwise, detect and track objects in the frames
    detections = detect_frames(model, frames)
    tracks = {"players": [], "referees": [], "ball": []}

    # Process each frame to generate tracks
    for frame_num, detection in enumerate(detections):
        cls_names = detection.names
        cls_names_inv = {v: k for k, v in cls_names.items()}
        detection_supervision = sv.Detections.from_ultralytics(detection)

        # Convert goalkeeper to player
        for object_ind, class_id in enumerate(detection_supervision.class_id):
            if cls_names[class_id] == "goalkeeper":
                detection_supervision.class_id[object_ind] = cls_names_inv["player"]

        # Track objects using ByteTrack
        detection_with_tracks = tracker.update_with_detections(detection_supervision)

        # Initialize empty tracks for the current frame
        tracks["players"].append({})
        tracks["referees"].append({})
        tracks["ball"].append({})

        # Assign tracks to players and referees
        for frame_detection in detection_with_tracks:
            bbox = frame_detection[0].tolist()
            cls_id = frame_detection[3]
            track_id = frame_detection[4]

            if cls_id == cls_names_inv['player']:
                tracks["players"][frame_num][track_id] = {"bbox": bbox}
            if cls_id == cls_names_inv['referee']:
                tracks["referees"][frame_num][track_id] = {"bbox": bbox}

        # Assign tracks to the ball
        for frame_detection in detection_supervision:
            bbox = frame_detection[0].tolist()
            cls_id = frame_detection[3]
            if cls_id == cls_names_inv['ball']:
                tracks["ball"][frame_num][1] = {"bbox": bbox}

    return tracks


In [18]:
def add_positions_to_tracks(tracks):
    """
    Add adjusted positions (center or foot position) to the tracks dictionary.

    Args:
        tracks (dict): A dictionary containing tracks for players, referees, and the ball.

    Returns:
        dict: The updated tracks dictionary with adjusted positions added.
    """
    # Iterate through each object (e.g., players, referees, ball) in the tracks dictionary
    for object, object_tracks in tracks.items():
        # Iterate through each frame for the current object
        for frame_num, track in enumerate(object_tracks):
            # Iterate through each track in the current frame
            for track_id, track_info in track.items():
                # Get the bounding box of the object
                bbox = track_info['bbox']

                # Calculate the adjusted position based on the object type
                if object == 'ball':
                    # For the ball, use the center of the bounding box
                    position = get_center_of_bbox(bbox)
                else:
                    # For players and referees, use the foot position (bottom-center of the bounding box)
                    position = get_foot_position(bbox)

                # Add the adjusted position to the tracks dictionary
                tracks[object][frame_num][track_id]['position_adjusted'] = position

    return tracks

In [19]:
def interpolate_ball_positions(ball_positions):
    """
    Interpolate missing ball positions in a sequence of ball position data.

    Args:
        ball_positions (list): A list of dictionaries containing ball position data.
                               Each dictionary may contain a key `1` with a nested dictionary
                               that includes a 'bbox' key representing bounding box coordinates.

    Returns:
        list: A list of dictionaries with interpolated ball positions.
    """
    ball_positions = [x.get(1, {}).get('bbox', []) for x in ball_positions]
    df_ball_positions = pd.DataFrame(ball_positions, columns=['x1', 'y1', 'x2', 'y2'])
    df_ball_positions = df_ball_positions.interpolate()
    df_ball_positions = df_ball_positions.bfill()
    ball_positions = [{1: {"bbox": x}} for x in df_ball_positions.to_numpy().tolist()]
    return ball_positions

In [20]:
def detect_frames(model, frames, batch_size=20):
    """
    Detect objects in a list of frames using the YOLO model.

    Args:
        model: The YOLO model used for object detection.
        frames (list): A list of frames (images) to process.
        batch_size (int, optional): The number of frames to process in one batch. Defaults to 20.

    Returns:
        list: A list of detections for all the frames.
    """
    detections = []
    for i in range(0, len(frames), batch_size):
        detections_batch = model.predict(frames[i:i + batch_size], conf=0.1)
        detections += detections_batch
    return detections

In [21]:
def add_speed_and_distance_to_tracks(tracks, frame_window=5, frame_rate=24):
    """
    Add speed and distance information to tracked objects in the given tracks.

    Args:
        tracks (dict): A dictionary containing tracks for players, referees, and the ball.
                       Each key maps to a list of dictionaries representing tracked objects in each frame.
        frame_window (int, optional): The number of frames over which to calculate speed and distance. Defaults to 5.
        frame_rate (int, optional): The frame rate of the video (frames per second). Defaults to 24.

    Returns:
        dict: Updated tracks dictionary with speed and distance information added for each tracked object.
    """
    total_distance = {}

    for object, object_tracks in tracks.items():
        if object == "ball" or object == "referees":
            continue

        number_of_frames = len(object_tracks)
        for frame_num in range(0, number_of_frames, frame_window):
            last_frame = min(frame_num + frame_window, number_of_frames - 1)

            for track_id, _ in object_tracks[frame_num].items():
                if track_id not in object_tracks[last_frame]:
                    continue

                start_position = object_tracks[frame_num][track_id].get('position_transformed', None)
                end_position = object_tracks[last_frame][track_id].get('position_transformed', None)

                if start_position is None or end_position is None:
                    # print(f"Skipping track {track_id} in frame {frame_num}: Missing position_transformed")
                    continue

                distance_covered = measure_distance(start_position, end_position)
                time_elapsed = (last_frame - frame_num) / frame_rate
                speed_meters_per_second = distance_covered / time_elapsed
                speed_km_per_hour = speed_meters_per_second * 3.6

                if object not in total_distance:
                    total_distance[object] = {}
                if track_id not in total_distance[object]:
                    total_distance[object][track_id] = 0

                total_distance[object][track_id] += distance_covered

                for frame_num_batch in range(frame_num, last_frame):
                    if track_id not in tracks[object][frame_num_batch]:
                        continue
                    tracks[object][frame_num_batch][track_id]['speed'] = speed_km_per_hour
                    tracks[object][frame_num_batch][track_id]['distance'] = total_distance[object][track_id]

                # Debug prints

    return tracks

In [22]:
def draw_speed_and_distance(frames, tracks):
    """
    Draw speed and distance information on frames for tracked objects.

    Args:
        frames (list): A list of frames (images) to process.
        tracks (dict): A dictionary containing tracks for players, referees, and the ball.
                       Each key maps to a list of dictionaries representing tracked objects in each frame.

    Returns:
        list: A list of output frames with speed and distance information drawn on them.
    """
    output_frames = []
    for frame_num, frame in enumerate(frames):
        for object, object_tracks in tracks.items():
            if object == "ball" or object == "referees":
                continue

            for track_id, track_info in object_tracks[frame_num].items():
                if "speed" in track_info:
                    speed = track_info.get('speed', None)
                    distance = track_info.get('distance', None)
                    # print(f"Frame {frame_num}, Object: {object}, Track ID: {track_id}, Speed: {speed}, Distance: {distance}")

                    if speed is None or distance is None:
                        continue

                    bbox = track_info['bbox']
                    position = get_foot_position(bbox)
                    position = list(position)
                    position[1] += 40  # Adjust position for text placement
                    position = tuple(map(int, position))

                    # Draw speed and distance on the frame
                    cv2.putText(frame, f"{speed:.2f} km/h", position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                    cv2.putText(frame, f"{distance:.2f} m", (position[0], position[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

        output_frames.append(frame)

    return output_frames

In [23]:
# Process Video for Players Detection and Tracking
def process_video_for_players_detection_and_tracking(roboflow_api_key, source_video_path, output_video_path, team_classifier, tracker):
    """
    Process a video for player detection, tracking, and analysis.

    Args:
        roboflow_api_key (str): API key for accessing the Roboflow model.
        source_video_path (str): Path to the input video file.
        output_video_path (str): Path to save the output video file.
        team_classifier: A pre-trained classifier for identifying teams.
        tracker: A tracker object for tracking detected objects across frames.

    Returns:
        None: The function saves the processed video to the specified output path.
    """
    frame_generator = sv.get_video_frames_generator(source_video_path)
    video_frames = [frame for frame in frame_generator]
    player_detection_model = load_models(roboflow_api_key)

    # Collect player crops and train the team classifier
    collect_and_train_team_classifier(source_video_path, player_detection_model, team_classifier)

        # Initialize Tracker
    model, tracker = initialize_tracker('/kaggle/input/yolov11l/pytorch/default/1/best.pt')
    tracks = get_object_tracks(model, tracker, video_frames)

    # Add positions to tracks
    tracks = add_positions_to_tracks(tracks)

    # Inside the main pipeline
    pixel_vertices = [[110, 1035], [265, 275], [910, 260], [1640, 915]]
    target_vertices = [[0, 68], [0, 0], [23.32, 0], [23.32, 68]]
    perspective_transformer = initialize_view_transformer(pixel_vertices, target_vertices)
    tracks = add_transformed_positions_to_tracks(tracks, pixel_vertices, perspective_transformer)

    # Initialize Camera Movement Estimator
    frame_generator = sv.get_video_frames_generator(source_video_path)
    frames = [frame for frame in frame_generator]
    lk_params, features, first_frame_grayscale = initialize_camera_movement_estimator(frames[0])
    camera_movement_per_frame = get_camera_movement(frames, lk_params, features)

    # Interpolate ball positions
    tracks["ball"] = interpolate_ball_positions(tracks["ball"])

    # Add speed and distance to tracks
    tracks = add_speed_and_distance_to_tracks(tracks)


    # Initialize Video Writer
    writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), video_info.fps, (video_info.width, video_info.height))
    if not writer.isOpened():
        print("Error: VideoWriter is not opened!")
        return


    # Second pass: Process video frames with team classification
    frame_generator = sv.get_video_frames_generator(source_video_path)  # Reinitialize frame generator
    for frame_num, frame in enumerate(tqdm(frame_generator, total=video_info.total_frames)):
        print(f"Processing frame {frame_num}")

        # Check if frame is valid
        if frame is None:
            print("Error: Frame is None!")
            break

        # Process frame
        annotated_frame, detections = process_frame_for_detection_and_tracking(frame, player_detection_model, team_classifier, tracker)
        

        if annotated_frame is None:
            print("Error: Annotated frame is None!")
            break
        print("Annotated frame shape:", annotated_frame.shape)

        # Draw camera movement on the frame
        annotated_frame = draw_camera_movement(annotated_frame, camera_movement_per_frame[frame_num])

        annotated_frame = draw_speed_and_distance([annotated_frame], {k: [v[frame_num]] for k, v in tracks.items()})[0]
        # Write annotated frame to output video
        writer.write(annotated_frame)
        print(f"Frame {frame_num} written to output video.")


    # Release Video Writer
    writer.release()
    print("Processing Complete. Output saved at:", output_video_path)

In [24]:
# Run the Pipeline
process_video_for_players_detection_and_tracking(ROBOFLOW_API_KEY, source_video_path, output_video_path, team_classifier, tracker) 

collecting crops:   2%|▏         | 15/750 [00:06<05:05,  2.41it/s]


Training Team Classifier...


Embedding extraction: 10it [00:02,  3.73it/s]


Team Classifier training complete.

0: 384x640 1 ball, 1 goalkeeper, 19 players, 3 referees, 10.0ms
1: 384x640 1 ball, 1 goalkeeper, 19 players, 3 referees, 10.0ms
2: 384x640 1 ball, 1 goalkeeper, 18 players, 3 referees, 10.0ms
3: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 10.0ms
4: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 10.0ms
5: 384x640 1 goalkeeper, 20 players, 3 referees, 10.0ms
6: 384x640 1 goalkeeper, 21 players, 3 referees, 10.0ms
7: 384x640 1 ball, 1 goalkeeper, 21 players, 3 referees, 10.0ms
8: 384x640 1 ball, 1 goalkeeper, 20 players, 3 referees, 10.0ms
9: 384x640 1 ball, 1 goalkeeper, 19 players, 3 referees, 10.0ms
10: 384x640 1 goalkeeper, 19 players, 3 referees, 10.0ms
11: 384x640 19 players, 3 referees, 10.0ms
12: 384x640 21 players, 3 referees, 10.0ms
13: 384x640 1 ball, 19 players, 3 referees, 10.0ms
14: 384x640 1 ball, 19 players, 3 referees, 10.0ms
15: 384x640 1 ball, 19 players, 3 referees, 10.0ms
16: 384x640 1 ball, 19 players, 3 referees, 

  0%|          | 0/750 [00:00<?, ?it/s]

...

Annotated frame shape: (1080, 1920, 3)
Frame 747 written to output video.
Processing frame 748



Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  5.67it/s]
100%|█████████▉| 749/750 [04:54<00:00,  2.59it/s]

Annotated frame shape: (1080, 1920, 3)
Frame 748 written to output video.
Processing frame 749



Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:00,  5.82it/s]
100%|██████████| 750/750 [04:55<00:00,  2.54it/s]

Annotated frame shape: (1080, 1920, 3)
Frame 749 written to output video.
Processing Complete. Output saved at: /kaggle/working/output_video.mp4
